<a href="https://colab.research.google.com/github/asabbah44/Text_Classification/blob/main/SVM_BOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.model_selection import train_test_split 
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfTransformer
from nltk import sent_tokenize
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
import string
from nltk.tokenize import word_tokenize
import xml

import csv
import ast
import logging
import pandas as pd
import numpy as np
from numpy import random
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix,recall_score ,precision_score, f1_score, average_precision_score
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
from google.colab import drive
%matplotlib inline
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
import statistics

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
drive.mount("/content/drive")   # Mount Googledrive

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/dataset.csv')
df2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/mydataset.csv')


df1 = pd.DataFrame(df1, columns = ['commenttext','label'])
df2 = pd.DataFrame(df2, columns = ['commenttext','label'])

data=pd.concat([df1,df2])

In [ ]:
data.head()

,commenttext,label
0,// FIXME formatters are not thread-safe,Defect
1,"// XXX: (Jon Skeet) The comment ""if it hasn't ...",Defect
2,"// I hate to admit it, but we don't know what ...",Defect
3,// Just a note: StarTeam has a status for NEW ...,Defect
4,// the generated classes must not be added in ...,Defect


In [ ]:
# data.drop(["Confident","ï»¿ID"],
#           axis=1,
#           inplace=True)


In [ ]:
data = np.asarray(data)

In [ ]:
Label = data[:,1]

In [ ]:
Label

array(['Defect', 'Defect', 'Defect', ..., 'Requirement', 'Defect',
       'Defect'], dtype=object)

In [ ]:
def labels_counters(data):
  RE= 0
  DEF = 0
  DES = 0
  DOC = 0
  TEST = 0
  Other = 0
 
  for i in data:
    if (i[1]== 'IMPLEMENTATION'):
      RE += 1
    if (i[1]== 'DEFECT'):
      DEF += 1
    if (i[1]== 'DESIGN'):
      DES += 1
    if (i[1]== 'DOCUMENTATION'):
      DOC += 1
    if (i[1]== 'TEST'):
      TEST += 1
    if (i[1]== ' '):
      Other += 1
  print("RE = {} , DEF = {}  , DES = {} , DOC = {} , TEST = {} , Other = {}".format(RE,DEF,DES,DOC,TEST,Other))

labels_counters(data)

  

RE = 0 , DEF = 0  , DES = 0 , DOC = 0 , TEST = 0 , Other = 0


In [ ]:
labels_counters(data)

RE = 0 , DEF = 0  , DES = 0 , DOC = 0 , TEST = 0 , Other = 0


In [ ]:
data[1][1]

'Defect'

In [ ]:
my_Label = ['RE ','DEF ','DES ','DOC ','TEST ','Other']

REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):

    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    text = re.sub('\d','', text)
    return text


def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
count = 0

def lematizer_sentence(sentence):
    # split into words
    tokenize = word_tokenize(sentence)
    
    # convert to lower case
    tokenize = [w.lower() for w in tokenize]
    
    # remove punctuation from each word
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokenize]
    
    # remove remaining tokens that are not alphabetic
    req_words = [word for word in stripped if word.isalpha()]
    stop_words = stopwords.words('english')
   
    # filter out stop words
    words = [w for w in req_words if not w in stop_words]
    
    
    words = [WordNetLemmatizer().lemmatize(word, get_wordnet_pos(word)) for word in words]

    return words
  

In [ ]:
df = pd.DataFrame(data= data,columns=["classification", "commenttext"])

In [ ]:
colors = tuple(["g", "SteelBlue","r","y","m","k"])

In [ ]:
df['commenttext'] = df['commenttext'].apply(clean_text)
df['commenttext'] = df['commenttext'].apply(lematizer_sentence)

In [ ]:
len(df['commenttext'])

5584

In [ ]:
import ast

In [ ]:
dec = []

In [ ]:
for sentence in df['commenttext']:
  for word in sentence:
   dec.append(word)



In [ ]:
len(dec)

5584

In [ ]:
dec.sort()

In [ ]:
dec = (list(dict.fromkeys(dec)))

In [ ]:
def generate_bow(req_sentences,dec):    
    # Define the BOW matrix
    bag_vector = np.zeros((len(req_sentences), len(dec)))
    # For each sentence
    for j in range(len(req_sentences)):
        # For each word within the sentence
        for w in req_sentences[j]:
            # For each word within the vocabulary
            for i,word in enumerate(dec):
                # If the word is in vocabulary, add 1 in position
                if word == w: 
                    bag_vector[j,i] += 1
    return bag_vector

In [ ]:
BOW = generate_bow(df['commenttext'],dec)




In [ ]:
print(len(BOW))
print(BOW)
print(df['commenttext'])

5584
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]
0            [defect]
1            [defect]
2            [defect]
3            [defect]
4            [defect]
            ...      
5579    [requirement]
5580         [design]
5581    [requirement]
5582         [defect]
5583         [defect]
Name: commenttext, Length: 5584, dtype: object


In [ ]:
from sklearn.svm import SVC

In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
import statistics
import matplotlib.pyplot as plt
% matplotlib inline

print(df.head(10))
plt.figure(figsize=(10,4))

plt.figure(figsize=(10,4))
df.classification.value_counts().plot(kind='bar',color=colors);

accuracy = []
recall = [] 
precision = []
f1 = []

recall_total= [] 
precision_total = []
f1_total = []



for i in range(0,10):
  x_train, x_test, y_train, y_test = train_test_split(BOW,Label, test_size=0.3)
  x_train = np.array(x_train)
  y_train = np.array(y_train)
  x_test = np.array(x_test)
  y_test = np.array(y_test)


  svm_model = SVC()

  svm_model.fit(x_train, y_train)
  y_pred = svm_model.predict(x_test)
  
  accuracy.append( accuracy_score(y_pred, y_test))
  print('accuracy %s' % accuracy_score(y_pred, y_test)) 
  precision.append(precision_score(y_pred, y_test,average=None,labels=my_Label))
  recall.append(recall_score(y_pred, y_test,average=None,labels=my_Label))
  f1.append(f1_score(y_pred, y_test,average=None,labels=my_Label))
  precision_total.append(precision_score(y_pred, y_test,average="micro"))
  recall_total.append(recall_score(y_pred, y_test,average='micro'))
  f1_total.append(f1_score(y_pred, y_test,average='micro'))


statistics.mean(accuracy)


  

In [ ]:
statistics.mean(accuracy)

In [ ]:
len(x_train)

In [ ]:
len(x_test)

In [ ]:
872 + 375

In [ ]:
print(classification_report(y_test, y_pred,target_names=my_Label))

In [ ]:
for x in accuracy:
  print (x)

In [ ]:
for x in precision_total:
  print(x)

print("Mean = ",statistics.mean(precision_total))  

In [ ]:
for x in recall_total:
  print(x)

print("Mean = ",statistics.mean(recall_total))  

In [ ]:
for x in f1_total:
  print(x)

print("Mean = ",statistics.mean(f1_total)) 

In [ ]:
def score_callculater(mat):
  
  all_class = []
  us = []
  for i in mat:
    us.append(i[0])
  all_class.append(statistics.mean(us))
  
  re = []
  for i in mat:
    re.append(i[1])
  all_class.append(statistics.mean(re) ) 

  pe = []
  for i in mat:
    pe.append(i[2])
  all_class.append(statistics.mean(pe)) 

  a = []
  for i in mat:
    a.append(i[3])
  all_class.append(statistics.mean(a)) 


  se = []
  for i in mat:
    se.append(i[4])
  all_class.append(statistics.mean(se))

  other = []
  for i in mat:
    other.append(i[5])
  all_class.append(statistics.mean(other))

  return all_class

In [ ]:
class_precision = score_callculater(precision)
class_recall = score_callculater(recall)
class_f1 = score_callculater(f1)

In [ ]:
for x in class_precision:
  print(x)

print("Avg ",statistics.mean(class_precision))  

In [ ]:
for x in class_recall:
  print(x)
print("Avg ",statistics.mean(class_recall))  

In [ ]:
for x in class_f1:
  print(x)
print("Avg ",statistics.mean(class_f1)) 

In [ ]:
x =0